In [76]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import os
import pandas as pd
import spacy
import src.preprocessing as processing
import src.lang as lang
import src.autoencoder as autoencoder
import src.autoencoder as train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [5]:
raw_data_path = os.path.join("data", "raw")
book_review_path = os.path.join(raw_data_path, os.path.join("reviews_Books_5.json", "Books_5.json"))
sports_review_path = os.path.join(raw_data_path, os.path.join("reviews_Sports_and_Outdoors_5.json", "Sports_and_Outdoors_5.json"))

In [6]:
full_dataset = pd.read_json(sports_review_path, lines=True)

In [7]:
full_dataset.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1881509818,"[0, 0]",5,This came in on time and I am veru happy with ...,"01 26, 2014",AIXZKN4ACSKI,David Briner,Woks very good,1390694400
1,1881509818,"[1, 1]",5,I had a factory Glock tool that I was using fo...,"02 2, 2012",A1L5P841VIO02V,Jason A. Kramer,Works as well as the factory tool,1328140800
2,1881509818,"[2, 2]",4,If you don't have a 3/32 punch or would like t...,"02 28, 2012",AB2W04NI4OEAD,J. Fernald,"It's a punch, that's all.",1330387200
3,1881509818,"[0, 0]",4,This works no better than any 3/32 punch you w...,"02 5, 2012",A148SVSWKTJKU6,"Jusitn A. Watts ""Maverick9614""",It's a punch with a Glock logo.,1328400000
4,1881509818,"[0, 0]",4,I purchased this thinking maybe I need a speci...,"04 23, 2013",AAAWJ6LW9WMOO,Material Man,"Ok,tool does what a regular punch does.",1366675200


In [8]:
NUM_EX = 100
oov_words, cleaned_dataset = processing.clean_data(full_dataset[:NUM_EX])

In [9]:
cleaned_dataset.reviewText[0].split(" ")

['This',
 'came',
 'in',
 'on',
 'time',
 'and',
 'I',
 'am',
 'veru',
 'happy',
 'with',
 'it,',
 'I',
 'haved',
 'used',
 'it',
 'already',
 'and',
 'it',
 'makes',
 'taking',
 'out',
 'the',
 'pins',
 'in',
 'my',
 'glock',
 '32',
 'very',
 'easy']

In [10]:
all_sentences =list(cleaned_dataset.reviewText.apply(lambda row : str.lower(row).strip().split(" ")))

In [51]:
vocab, idx_sentences = lang.custom_word_indices(all_sentences)

idx_to_word = {vocab[key] : key for key in vocab.keys()}

## Load Model

In [52]:
INPUT_SIZE = 10
HIDDEN_SIZE = 16
NUM_WORDS = len(vocab.keys())

encoder = autoencoder.Encoder(NUM_WORDS, INPUT_SIZE, HIDDEN_SIZE)
decoder = autoencoder.Decoder(NUM_WORDS, NUM_WORDS, HIDDEN_SIZE)

## Train

In [65]:
NUM_EPOCHS = 10
train_set = idx_sentences[:10]
all_losses, gen_s = train.train_dataset(train_set, encoder, decoder, 
                                 learning_rate = 0.01, epochs = NUM_EPOCHS)

In [73]:
print("Target Sentence is : {} \n".format(" ".join(all_sentences[0])))

for epoch_num in range(NUM_EPOCHS):
    print("{0} : {1} \n".format(epoch_num, " ".join([idx_to_word[idx] for idx in gen_s[epoch_num][0]])))

Target Sentence is : this came in on time and i am veru happy with it, i haved used it already and it makes taking out the pins in my glock 32 very easy 

0 : this a the the the and the a this a a the the a the a is to the is a the and a a the the the the and 

1 : this and the a to the the to the the to the the to the the to the the to the the to the the to the the to the 

2 : this and it the the the the a it it the the i a it it a this the is a the and to in the the the the the 

3 : this and the the the and the a it it the the i a it it a to it and it the the to in the the the i the 

4 : this this i the the the the the it this the the i a it it it to it and it the the the in the the the i the 

5 : this a and the a and the a and the a and the a and the a and the a and the a and the a and the a and 

6 : this this i the the i i the it it the the i a it it it it it it it the the the in the the the i the 

7 : this and it and the a to the the the the the the the the the the the the t